<a href="https://colab.research.google.com/github/sayakpaul/TF-2.0-Hacks/blob/master/Prototypical_Networks_in_TensorFlow_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data gathering

In [0]:
!wget https://github.com/brendenlake/omniglot/raw/master/python/images_background.zip

In [0]:
!wget https://github.com/brendenlake/omniglot/raw/master/python/images_evaluation.zip

In [0]:
!unzip images_background.zip

In [0]:
!unzip images_evaluation.zip

In [0]:
from matplotlib import pyplot as plt
import multiprocessing as mp
import pandas as pd 
import numpy as np 
import cv2
import os

## Data reading and augmentation
The Omniglot data set is designed for developing more human-like learning algorithms. It contains 1623 different handwritten characters from 50 different alphabets. Then to increase the number of classes, all the images are rotated by 90, 180 and 270 degrees and each rotation resulted in one more class. Hence the total count of classes reached to 6492(1623 * 4) classes. We split images of 4200 classes to training data and the rest went to test set.

In [0]:
train_dir = os.listdir('images_background/')
datax = np.array([])

In [0]:
def image_rotate(img, angle):
    """
    Image rotation at certain angle. It is used for data augmentation 
    """
    rows,cols, _ = img.shape
    M = cv2.getRotationMatrix2D((cols/2 ,rows/2),angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    return np.expand_dims(dst, 0)

In [0]:
def read_alphabets(alphabet_directory, directory):
    """
    Reads all the characters from alphabet_directory and augment each image with 90, 180, 270 degrees of rotation.
    """
    datax = None
    datay = []
    characters = os.listdir(alphabet_directory)
    for character in characters:
        images = os.listdir(alphabet_directory + character + '/')
        for img in images:
            image = cv2.resize(cv2.imread(alphabet_directory + character + '/' + img), (28,28))
            image90 = image_rotate(image, 90)
            image180 = image_rotate(image, 180)
            image270 = image_rotate(image, 270)
            image = np.expand_dims(image, 0)
            if datax is None:
                datax = np.vstack([image, image90, image180, image270])
            else:
                datax = np.vstack([datax, image, image90, image180, image270])
            datay.append(directory + '_' + character + '_0')
            datay.append(directory + '_' + character + '_90')
            datay.append(directory + '_' + character + '_180')
            datay.append(directory + '_' + character + '_270')
    return datax, np.array(datay)

In [0]:
def read_images(base_directory):
    """
    Used multithreading for data reading to decrease the reading time drastically
    """
    datax = None
    datay = []
    pool = mp.Pool(mp.cpu_count())
    results = [pool.apply(read_alphabets, args=(base_directory + '/' + directory + '/', directory, )) for directory in os.listdir(base_directory)]
    pool.close()
    for result in results:
        if datax is None:
            datax = result[0]
            datay = result[1]
        else:
            datax = np.vstack([datax, result[0]])
            datay = np.concatenate([datay, result[1]])
    return datax, datay

In [0]:
%time trainx, trainy = read_images('images_background/')

CPU times: user 783 ms, sys: 464 ms, total: 1.25 s
Wall time: 16.6 s


In [0]:
%time testx, testy = read_images('images_evaluation/')

CPU times: user 348 ms, sys: 346 ms, total: 694 ms
Wall time: 10.9 s


In [0]:
assert(trainx.shape[0] == trainy.shape[0])
assert(testx.shape[0] == testy.shape[0])

## Building the network in TensorFlow 2.0 using `tf.keras`

In [0]:
!pip install tensorflow-gpu==2.0.0-beta1

In [0]:
import tensorflow as tf

print(tf.__version__)

2.0.0-beta1


In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import pairwise_distances
from tensorflow import keras

In [0]:
class Net(keras.Model):
    """
    Image2Vector CNN which takes image of dimension (28x28x3) and return column vector length 64
    """
    def sub_block(self, out_channels=64, kernel_size=(3,3)):
        block = keras.models.Sequential([
                    keras.layers.Conv2D(filters=out_channels, \
                        kernel_size=kernel_size, input_shape=(28, 28, 3)),
                    keras.layers.BatchNormalization(),
                    keras.layers.ReLU(),
                    keras.layers.MaxPool2D(pool_size=(2,2))
        ])
        return block
    
    def __init__(self):
        super(Net, self).__init__()
        self.convnet1 = self.sub_block()
        self.convnet2 = self.sub_block()
        self.convnet3 = self.sub_block()
        self.convnet4 = self.sub_block()

    def call(self, x):
        x = self.convnet1(x)
        x = self.convnet2(x)
        x = self.convnet3(x)
        x = self.convnet4(x)
        x = keras.layers.Flatten()(x)
        return x

To be continued.

https://blog.floydhub.com/n-shot-learning/